# Data Preprocessing - Pipeline

## Import Packages
pandas: A library for analyzing, cleaning, and manipulating structured data using DataFrames and Series. \
os: A module for file system operations like creating, deleting, or navigating files and directories. \
glob: A module to find file paths matching patterns (e.g., *.csv) for handling multiple files easily.

In [13]:
import pandas as pd
import os
from glob import glob

## Companies Preprocessing


In [14]:
def preprocess_companies(companies_files):
    # Initialize a list to hold all processed company data frames
    all_companies = []

    # Loop through each file in the list of company files as we did multiple csv-exports from crunchbase.com
    for file in companies_files:
        # Read the CSV file into a DataFrame
        companies_df = pd.read_csv(file)

        # Columns to drop as they are not needed for our classification task
        drop_columns = [
            'Total Funding Amount (in USD)', 'Organization Name URL', 'Operating Status', 'Stage',
            'Headquarters Regions', 'Closed Date', 'Closed Date Precision', 'Company Type',
            'Announced Date Precision', 'Price', 'Price Currency', 'Price (in USD)',
            'Acquisition Terms', 'Money Raised at IPO', 'Money Raised at IPO Currency',
            'Money Raised at IPO (in USD)', 'Valuation at IPO', 'Valuation at IPO Currency',
            'Valuation at IPO (in USD)', 'Stock Symbol', 'Stock Symbol URL', 'Stock Exchange',
            'Founded Date Precision', 'Exit Date Precision', 'Number of Events', 'IPO Date',
            'Transaction Name', 'Transaction Name URL', 'Estimated Revenue Range',
            'Last Funding Amount Currency', 'Last Equity Funding Amount Currency',
            'Total Equity Funding Amount Currency', 'Total Funding Amount Currency',
            'Last Funding Amount (in USD)', 'Last Equity Funding Amount', 'Last Equity Funding Amount (in USD)',
            'Last Equity Funding Type', 'Total Equity Funding Amount', 'Total Equity Funding Amount (in USD)',
            'Funding Status', 'Acquired by URL', 'Acquisition Type', 'Acquired by', 'Announced Date Acquisition'
        ]

        companies_df.drop(columns=drop_columns, inplace=True, errors='ignore')

        # Handle missing values in the 'Number of Founders' column if it exists with filling in the mean value
        if 'Number of Founders' in companies_df.columns:
            mean_founders = companies_df['Number of Founders'].mean()
            companies_df['Number of Founders'] = companies_df['Number of Founders'].fillna(mean_founders)

        # Drop rows where 'Number of Employees' is missing
        companies_df = companies_df.dropna(subset=['Number of Employees'])

        # Convert 'Industries' and 'Industry Groups' into a list
        for col in ['Industries', 'Industry Groups']:
            if col in companies_df.columns:
                companies_df[col] = companies_df[col].fillna('').apply(lambda x: [item.strip() for item in x.split(',')])

        # Function to convert employee range strings into the mean of the range
        def range_to_mean(value):
            try:
                low, high = map(int, value.split('-'))  # Split the range into low and high values
                return (low + high) / 2  # Calculate the mean of the range
            except ValueError:
                return None

        companies_df['Number of Employees'] = companies_df['Number of Employees'].apply(range_to_mean)

        # Rename 'Announced Date' column to 'Announced Date Acquisition'
        if 'Announced Date' in companies_df.columns:
            companies_df.rename(columns={'Announced Date': 'Announced Date Acquisition'}, inplace=True)

        # Convert date columns to datetime format
        for date_col in ['Founded Date', 'Exit Date', 'Last Funding Date', 'Announced Date Acquisition']:
            if date_col in companies_df.columns:
                companies_df[date_col] = pd.to_datetime(companies_df[date_col], errors='coerce')

        # Map 'IPO Status' to a binary indicator (1 for 'Public', 0 for 'Private') and drop the original column
        if 'IPO Status' in companies_df.columns:
            companies_df['IPO'] = companies_df['IPO Status'].map({'Public': 1, 'Private': 0})
            companies_df.drop(columns=['IPO Status'], inplace=True)

        # Fill missing values in numeric columns with 0
        for column in ['Number of Lead Investors', 'Number of Investors', 'Number of Acquisitions']:
            if column in companies_df.columns:
                companies_df[column] = companies_df[column].fillna(0)

        # Append the processed DataFrame to the list
        all_companies.append(companies_df)

    # Combine all processed DataFrames into one
    merged_companies = pd.concat(all_companies, ignore_index=True)

    # Remove duplicate entries based on 'Organization Name', keeping the first occurrence
    merged_companies = merged_companies.drop_duplicates(subset=['Organization Name'], keep='first')

    # Add a unique identifier ('ID') for each row, zero-padded to 6 digits
    merged_companies['ID'] = (merged_companies.index + 1).astype(str).str.zfill(6)

    # Rearrange columns to ensure 'ID' is the first column
    merged_companies = merged_companies[['ID'] + [col for col in merged_companies.columns if col != 'ID']]

    # Return the final processed DataFrame
    return merged_companies

## Funding Preprocessing

In [15]:
def preprocess_funding(funding_files, companies_df):
    # Combine all funding csv files into a single data frame
    all_funding = pd.concat([pd.read_csv(file) for file in funding_files], ignore_index=True)

    # Columns to drop from the funding data frame as they are not needed
    drop_columns = [
        'Transaction Name', 'Transaction Name URL', 'Organization Name URL', 'Money Raised Currency',
        'Pre-Money Valuation Currency', 'Diversity Spotlight', 'Organization Location', 'Money Raised (in USD)',
        'Pre-Money Valuation (in USD)', 'Organization Description', 'Organization Industries',
        'Organization Website', 'Organization Revenue Range', 'Number of Partner Investors',
        'CB Rank (Funding Round)', 'Total Funding Amount Currency', 'Total Funding Amount (in USD)',
        'Funding Status', 'Equity Only Funding', 'Pre-Money Valuation', 'Total Funding Amount',
        'Number of Funding Rounds', 'Funding Stage'
    ]

    all_funding.drop(columns=drop_columns, inplace=True)

    # Merge the funding data with the companies DataFrame to add 'ID' (Company ID)
    all_funding = all_funding.merge(
        companies_df[['Organization Name', 'ID']],
        on='Organization Name', how='left'
    )

    # Rename 'ID' to 'Company ID' for clarity
    all_funding.rename(columns={'ID': 'Company ID'}, inplace=True)

    # Fill missing 'Company ID' values with a default '000000'
    all_funding['Company ID'] = all_funding['Company ID'].fillna('000000').astype(str).str.zfill(6)

    # Process the 'Investor Names' column:
    # If missing, fill with an empty string and convert comma-separated values into a list of trimmed names
    all_funding['Investor Names'] = all_funding['Investor Names'].fillna('').apply(
        lambda x: [item.strip() for item in x.split(',')]
    )

    # Convert 'Announced Date' to datetime format
    all_funding['Announced Date'] = pd.to_datetime(all_funding['Announced Date'])

    all_funding = all_funding.reset_index(drop=True)

    # Create a unique identifier ('Funding ID') for each funding record, zero-padded to 6 digits
    all_funding['Funding ID'] = (all_funding.index + 1).astype(str).str.zfill(6)

    # Rearrange columns to ensure 'Funding ID' is the first column
    all_funding = all_funding[['Funding ID'] + [col for col in all_funding.columns if col != 'Funding ID']]

    return all_funding

## Investors Preprocessing

In [16]:
def preprocess_investors(investor_files):

    # Initialize a list to store processed DataFrames from each file
    investor_dfs = []

    # Loop through each file in the list of investor files
    for file in investor_files:

        df = pd.read_csv(file)

        # Extract the country code from the file name (assuming the format: <prefix>-<country>.csv)
        country = os.path.basename(file).split('-')[1].split('.')[0].upper()

        # Add a 'Country' column to the data frame with the extracted country code
        df['Country'] = country

        # Append the processed DataFrame to the list
        investor_dfs.append(df)

    # Combine all individual investor DataFrames into one consolidated data frame
    investors = pd.concat(investor_dfs, ignore_index=True)

    # Drop unnecessary columns to clean up the data
    investors.drop(
        columns=['Organization/Person Name URL', 'Investment Stage', 'Regions', 'Founded Date Precision'],
        inplace=True
    )

    # Generate a unique identifier for each investor, start from 1 and format as zero-padded strings with 6 digits
    investors['Investor ID'] = range(1, len(investors) + 1)
    investors['Investor ID'] = investors['Investor ID'].apply(lambda x: str(x).zfill(6))

    # Reorder columns so 'Investor ID' is the first column
    cols = ['Investor ID'] + [col for col in investors.columns if col != 'Investor ID']
    investors = investors[cols]

    return investors

## Founders Preprocessing

In [17]:
def process_founders(companies, people):
    # Define required columns for the people dataset
    required_columns = ['first_name', 'last_name', 'linkedin_url', 'logo_url', 'facebook_url',
                        'twitter_url', 'city', 'region', 'country_code', 'featured_job_title',
                        'featured_job_organization_name']

    # Check for missing columns in the people dataset
    missing_columns = [col for col in required_columns if col not in people.columns]
    if missing_columns:
        raise ValueError(f"Missing columns in 'people': {missing_columns}")

    # Create a copy of the companies DataFrame to avoid modifying the original
    temp_companies = companies.copy()

    # Process the 'Founders' column, converting it into a list of names
    temp_companies['Founders'] = temp_companies['Founders'].fillna('').astype(str)
    temp_companies['Founders'] = temp_companies['Founders'].apply(
        lambda x: [item.strip() for item in x.split(',') if item.strip()]
    )

    # Explode the 'Founders' column so that each founder becomes a separate row
    founders_table = temp_companies.explode('Founders', ignore_index=True)

    # Create a new data frame with relevant columns
    founders = founders_table[['ID', 'Founders', 'Organization Name']].rename(
        columns={'ID': 'Company ID', 'Founders': 'Founder Name'}
    )

    # Generate unique Founder IDs
    founders['Founder ID'] = founders.index + 1
    founders['Founder ID'] = founders['Founder ID'].astype(str).str.zfill(6)

    # Rearrange and drop rows where 'Founder Name' is missing
    founders = founders[['Founder ID', 'Company ID', 'Founder Name', 'Organization Name']]
    founders = founders.dropna(subset=['Founder Name'])

    # Split 'Founder Name' into 'first_name' and 'last_name'
    founders[['first_name', 'last_name']] = founders['Founder Name'].str.split(' ', n=1, expand=True)

    # Add a placeholder column for LinkedIn URL
    founders['linkedin_url'] = None

    # Add and initialize additional string-based columns
    string_columns = ['logo_url', 'facebook_url', 'twitter_url', 'city', 'region',
                      'country_code', 'featured_job_title', 'featured_job_organization_name']
    for col in string_columns:
        if col not in founders.columns:
            founders[col] = None
        founders[col] = founders[col].astype('str')

    # Match each founder with their data from the 'people' dataset
    for idx, row in founders.iterrows():
        matches = people[
            (people['first_name'] == row['first_name']) &
            (people['last_name'] == row['last_name'])
            ]
        if len(matches) == 1:
            match = matches.iloc[0]
            founders.at[idx, 'linkedin_url'] = match['linkedin_url']  # Update LinkedIn URL
            for col in string_columns:  # Update other details from the match
                founders.at[idx, col] = match[col]

    return founders

def enrich_founders_with_linkedin(founders, founders_linkedin):
    # Define required columns in the LinkedIn dataset
    required_columns = ['url', 'followers', 'connections', 'degree_1', 'degree_1_university',
                        'degree_2', 'degree_2_university']

    # Check for missing columns in the LinkedIn dataset
    missing_columns = [col for col in required_columns if col not in founders_linkedin.columns]
    if missing_columns:
        raise ValueError(f"Missing columns in 'founders_linkedin': {missing_columns}")

    # Rename 'url' column to 'linkedin_url' for consistency with the founders DataFrame
    founders_linkedin = founders_linkedin.rename(columns={'url': 'linkedin_url'})

    # Merge the LinkedIn data with the founders data on 'linkedin_url'
    enriched_founders = founders.merge(founders_linkedin, on='linkedin_url', how='left')

    return enriched_founders

def process_and_enrich_founders(companies, people, founders_linkedin):
    # Process founder data from companies and people datasets
    founders = process_founders(companies, people)

    # Enrich processed founder data with LinkedIn information
    enriched_founders = enrich_founders_with_linkedin(founders, founders_linkedin)

    # Remove duplicate founders, keeping only the first occurrence
    enriched_founders = enriched_founders.drop_duplicates(subset=['Founder Name'], keep='first')

    return enriched_founders

## Pipeline Execution
Execute the defined functions on the saved data sets.

In [18]:
companies_files = glob('../../Datasets/Companies/*.csv')
funding_files = glob('../../Datasets/Funding/*.csv')
investor_files = glob('../../Datasets/Investors/*.csv')

companies = preprocess_companies(companies_files)
funding = preprocess_funding(funding_files, companies)
investors = preprocess_investors(investor_files)

people = pd.read_csv("../../Datasets/People/people_CB.csv")

founders_linkedin = pd.read_csv("../../Datasets/LinkedIn/Founders/founders_linkedin.csv")

founders = process_and_enrich_founders(companies, people, founders_linkedin)

## LinkedIn-Company Data

In [19]:
# Define the file path to LinkedIn CSV files using a wildcard to match all .csv files
linkedin_files_path = '/../../Code/Social Media Webscraping/Results/LinkedIn/*.csv'

# Use glob to get a list of all LinkedIn CSV file paths matching the specified pattern
linkedin_files = glob(linkedin_files_path)

# Read all LinkedIn company CSV files into a list of data frame
linkedin_companies = [pd.read_csv(file) for file in linkedin_files]

# Combine all LinkedIn company data frame into a single data frame
linkedin_companies_combined = pd.concat(linkedin_companies, ignore_index=True)

# Remove leading and trailing whitespace from the 'url' column in the LinkedIn data frame
linkedin_companies_combined['url'] = linkedin_companies_combined['url'].str.strip()

# Remove leading and trailing whitespace from the 'LinkedIn' column in the companies data frame
companies['LinkedIn'] = companies['LinkedIn'].str.strip()

# Drop unnecessary columns from the LinkedIn companies data frame
linkedin_companies_combined = linkedin_companies_combined.drop(columns=['company_id', 'error'], errors='ignore')

# Merge the companies data frame with the LinkedIn companies data frame
# Match rows based on the 'LinkedIn' column in companies and the 'url' column in LinkedIn companies
companies = companies.merge(
    linkedin_companies_combined,
    left_on='LinkedIn',
    right_on='url',
    how='left',
    suffixes=('', '_LinkedIn')
)

# This column is redundant since it was used as the join key
companies = companies.drop(columns=['url'], errors='ignore')

## Twitter / X Company Data

In [20]:
# Define the file path to Twitter CSV files using a wildcard to match all .csv files
twitter_files_path = '/../../Code/Social Media Webscraping/Results/Twitter/*.csv'

twitter_files = glob(twitter_files_path)

# Read each Twitter company CSV file into a list of data frames and combine it to one data frame
twitter_companies = [pd.read_csv(file) for file in twitter_files]

twitter_companies_combined = pd.concat(twitter_companies, ignore_index=True)

# Remove leading and trailing whitespace from the 'url' column in the Twitter data frame
twitter_companies_combined['url'] = twitter_companies_combined['url'].str.strip()

# Remove leading and trailing whitespace from the 'Twitter' column in the companies DataFrame
companies['Twitter'] = companies['Twitter'].str.strip()

# Drop unnecessary columns from the Twitter companies DataFrame
twitter_companies_combined = twitter_companies_combined.drop(columns=['company_id', 'error'], errors='ignore')

# Merge the companies DataFrame with the Twitter companies data frame
companies = companies.merge(
    twitter_companies_combined,
    left_on='Twitter',
    right_on='url',
    how='left',
    suffixes=('', '_Twitter')
)

companies.drop(columns=['url'], inplace=True, errors='ignore')

## Validation

In [21]:
founders

,Founder ID,Company ID,Founder Name,Organization Name,first_name,last_name,linkedin_url,logo_url,facebook_url,twitter_url,...,featured_job_title,featured_job_organization_name,username,followers,connections,degree_1,degree_1_university,degree_2,degree_2_university,error
0,000001,000001,Johannes Stoffel,2trde,Johannes,Stoffel,https://www.linkedin.com/in/johannes-stoffel-2...,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,Co-Founder & CEO,Karosso,johannes-stoffel-27389667,4815.0,3914.0,"Executive Master, Digital Innovation and Entre...",ESCP Business School,"Bachelor of Science - BS, Gerneral Management",EBS Universität für Wirtschaft und Recht,NaN
2,000002,000002,Florian Ziesche,36ZERO Vision,Florian,Ziesche,None,None,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000003,000002,Zeeshan Karamat,36ZERO Vision,Zeeshan,Karamat,https://www.linkedin.com/in/zkaramat/,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,Chief Technology Officer & Co-Founder,36ZERO Vision,zkaramat,6482.0,6479.0,"Masters Computer Science, Artificial Intelligence",Technische Universität München,"Masters Computer Science, Artificial Intelligence",Georgia Institute of Technology,NaN
4,000004,000003,Caroline Steingruber,3Bears Foods,Caroline,Steingruber,None,None,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,000005,000003,Tim Nichols,3Bears Foods,Tim,Nichols,None,None,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3013,003137,001508,Sven Peper,Taxy.io,Sven,Peper,https://www.linkedin.com/in/sven-peper,NaN,NaN,NaN,...,CEO & Co-Founder,Taxy.io,sven-peper,1275.0,1174.0,Ingenieurwissenschaften,RWTH Aachen University,NaN,NaN,NaN
3014,003138,001508,Sven Weber,Taxy.io,Sven,Weber,http://www.linkedin.com/in/svenweber,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,Managing Principal,Knightsbridge Advisers LLC,svenweber,5489.0,5477.0,"Master, Physics",Universität Heidelberg,NaN,NaN,NaN
3015,003139,001509,Dr. Gennadi Schechtmann,TRINKKOST GmbH,Dr.,Gennadi Schechtmann,None,None,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3016,003140,001509,Timon Ortloff,TRINKKOST GmbH,Timon,Ortloff,None,None,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
companies

,ID,Organization Name,Industries,Headquarters Location,Description,CB Rank (Company),Postal Code,Founded Date,Exit Date,Website,...,followers_Twitter,following,tweets,account_created,account_age_days,tweet_activity,followers_max_growth,followers_max_loss,tweets_max_growth,tweets_max_loss
0,000001,2trde,"[Automotive, Software]","Munich, Bayern, Germany",2trde develops a software solution designed fo...,"57,123",NaN,2017-01-01,NaT,https://www.2trde.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000002,36ZERO Vision,"[Artificial Intelligence (AI), Computer Vision...","Munich, Bayern, Germany",36ZEROVision is an AI-powered visual inspectio...,"51,326",81671,2019-01-01,NaT,https://36zerovision.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000003,3Bears Foods,[Food and Beverage],"Munich, Bayern, Germany",3Bears Foods enable a balanced and delicious b...,"275,817",NaN,2015-01-01,NaT,https://3bears.de/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000004,3dTrust,"[3D Printing, Manufacturing, Software]","Munich, Bayern, Germany",3dTrust helps companies integrate 3D printing ...,"134,694",80797,2015-01-01,NaT,http://3dtrust.de,...,269.0,346.0,248.0,2015-09-23,3389.0,0.07,2.0,-3.0,0.0,-2.0
4,000005,abaut,"[Analytics, Artificial Intelligence (AI), Cons...","Munich, Bayern, Germany",abaut builds a SaaS that enables businesses al...,"219,525",80992,2017-07-21,NaT,https://abaut.de,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,001506,SAYM,"[Apps, B2B, B2C, Human Resources, Mobile Apps,...","Aachen, Nordrhein-Westfalen, Germany",The SAYM platform for swarm mobility defines t...,"166,910",52070.0,2019-01-01,NaT,https://www.saym.io/,...,102.0,438.0,169.0,2018-03-01,2499.0,0.07,4.0,-2.0,0.0,-2.0
1514,001507,SONAH GmbH,"[Apps, Artificial Intelligence (AI), Computer ...","Aachen, Nordrhein-Westfalen, Germany",SONAH developed a flexible embedded vision sen...,"121,752",52070.0,2016-01-01,NaT,http://www.sonah.tech,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1515,001508,Taxy.io,"[FinTech, Legal Tech, Machine Learning, Software]","Aachen, Nordrhein-Westfalen, Germany",Taxy.io builds the leading platform for B2B ta...,"220,816",52070.0,2019-01-01,NaT,https://www.taxy.io/,...,232.0,482.0,786.0,2018-04-02,2467.0,0.32,29.0,-42.0,104.0,-8.0
1516,001509,TRINKKOST GmbH,"[Agriculture, Consumer Goods, Fitness, Food an...","Aachen, Nordrhein-Westfalen, Germany",TRINKKOST is a food supplement manufacturing c...,"907,817",NaN,2016-01-01,NaT,http://www.trinkkost.de,...,69.0,298.0,71.0,2015-07-18,3456.0,0.02,4.0,-4.0,0.0,-1.0


In [23]:
funding

,Funding ID,Organization Name,Funding Type,Money Raised,Announced Date,Lead Investors,Number of Investors,Investor Names,Company ID
0,000001,4stop,Series A,2500000.0,2019-05-15,Ventech,1.0,[Ventech],000000
1,000002,aiconix GmbH,Convertible Note,NaN,2019-06-17,NaN,NaN,[],001193
2,000003,aiconix GmbH,Pre-Seed,300000.0,2019-08-28,NaN,NaN,[],001193
3,000004,AutLay,Seed,NaN,2019-05-22,Crew Ventures,1.0,[Crew Ventures],001196
4,000005,Buynomics,Pre-Seed,NaN,2019-09-01,DvH Ventures,2.0,"[DvH Ventures, Tomahawk.VC]",001205
...,...,...,...,...,...,...,...,...,...
6863,006864,VAMOS.ai,Seed,NaN,2020-01-01,DDG AG,1.0,[DDG AG],000000
6864,006865,WeProfit,Pre-Seed,272000.0,2021-05-28,NaN,5.0,"[Ara Abrahamyan, Armen Kocharyan, Clemens Boll...",000298
6865,006866,WindStar Medical GmbH,Seed,NaN,2020-12-07,Project A Ventures,1.0,[Project A Ventures],000000
6866,006867,WorkMentality Foundation,Seed,NaN,2023-03-01,NaN,1.0,[Deutsche Bank],000000


In [24]:
investors

,Investor ID,Organization/Person Name,Investor Type,Number of Investments,Number of Exits,Location,Description,Founded Date,Website,LinkedIn,Number of Portfolio Organizations,Number of Lead Investments,Number of Exits (IPO),Country
0,000001,Christian Edler,"Individual/Angel, Investment Partner",57,10.0,"Berlin, Berlin, Germany",Christian O. Edler is an investor and entrepre...,NaN,http://www.christianedler.com,https://www.linkedin.com/in/christianedler/,50,NaN,NaN,GER
1,000002,Ralf Dummel,Individual/Angel,38,NaN,"Hamburg, Hamburg, Germany","Ralf Dummel, born on 02.12.1966 in Bad Segeber...",NaN,NaN,NaN,38,13.0,NaN,GER
2,000003,Carsten Maschmeyer,"Individual/Angel, Investment Partner",36,3.0,"Hanover, Niedersachsen, Germany",Carsten Maschmeyer is a successful entrepreneu...,NaN,NaN,https://www.linkedin.com/in/carsten-maschmeyer/,34,8.0,NaN,GER
3,000004,Kai Hansen,"Individual/Angel, Investment Partner",36,4.0,"Berlin, Berlin, Germany",business angel // entrepreneur // founder lief...,NaN,NaN,https://www.linkedin.com/in/hansenkai/en,25,NaN,NaN,GER
4,000005,Chris Schagen,Individual/Angel,33,4.0,"Berlin, Berlin, Germany",I help developer tech entrepreneurs beginning ...,NaN,NaN,https://www.linkedin.com/in/christianschagen/,29,NaN,NaN,GER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,008210,Manchester Technology Fund,Venture Capital,3,1.0,"Manchester, Manchester, United Kingdom","Manchester Technology Fund, a Manchester-based...",1999-01-01,http://www.mantechfund.com,NaN,3,NaN,NaN,UK
8210,008211,British Smaller Technology Companies VCT 2,Venture Capital,3,1.0,"Leeds, Leeds, United Kingdom","British Smaller Technology Companies VCT 2, a ...",NaN,NaN,NaN,3,NaN,NaN,UK
8211,008212,Elwin Capital Partners,Venture Capital,3,2.0,"London, England, United Kingdom",Elwin Capital Partners is out of business. It ...,2000-01-01,http://www.elwincapital.com/,NaN,3,1.0,NaN,UK
8212,008213,Puma Investments,Venture Capital,3,NaN,"London, England, United Kingdom",Puma Investments is a investment firm that pro...,2012-01-01,https://www.pumainvestments.co.uk/,https://www.linkedin.com/company/puma-investme...,2,2.0,NaN,UK
